SPLIT DATA (WINDOWING)

In [ ]:
from src.execution import WindowSplittingExecution

WINDOW SPLIT

In [ ]:
DATA: str = "assets/stationary_data_250403_01/sim_data_preprocessed_offset_multiplitiv_with_rot_forces.csv"
VALIDATION_SPLIT: bool = True # If the data should also be splitted into a validation set?
TEST_SIZE: float = 0.2
BATCH_SPLIT: bool = True
BATCHSIZE: int = 326 # real-data: 1800, synthetic-data: 326
SEED: int = 278 # Seed for random state -> Split with same seed and data will always result in the same split
INTERPOLATION: bool = False
WINDOWSIZE: int = 10
SEP: str = ";" # Separator of the csv file
DECIMAL: str = "." # Decimal separator of the csv file


WindowSplittingExecution.execute(DATA, 
                                 BATCH_SPLIT, 
                                 VALIDATION_SPLIT, 
                                 TEST_SIZE, 
                                 SEED, 
                                 BATCHSIZE, 
                                 INTERPOLATION, 
                                 WINDOWSIZE, 
                                 SEP, 
                                 DECIMAL)

TUNE MODEL

In [ ]:
from src.execution import CNNTuningExecution

In [ ]:
SPLITTED_DATA_FOLDER = "build/window_split/sim_data_preprocessed_offset_multiplitiv_with_rot_forces/1743718326"

CNNTuningExecution.execute(SPLITTED_DATA_FOLDER)

TRAIN MODEL

In [ ]:
from src.execution import CNNTrainingExecution

In [ ]:
TUNED_MODEL_FILE = "build/tune/real-data/1743684695/best-model.h5"
DATA_FOLDER = "build/window_split/sim_data_preprocessed/1743520007"
SAVE_FILENMAME = "cnn-preprocessed.h5" # With file extension
LEARNING_RATE = 0.007744543497026102 # !!!!

CNNTrainingExecution.execute(TUNED_MODEL_FILE, DATA_FOLDER, SAVE_FILENMAME, LEARNING_RATE)

VALIDATE MODEL

In [ ]:
from src.execution import CNNValidationExecution 

In [ ]:
MODEL_FOLDER = "assets/models/corvin/cnn-corvin-60-trials.h5"
DATA_FOLDER = "build/window_split/real-data/1743522733"

CNNValidationExecution.execute(MODEL_FOLDER, DATA_FOLDER)

In [ ]:
from src.execution import CNNSoftStartExecution

In [ ]:
MODEL_FILE = "build/train/cnn-synthetic-data-offset/1742244525/cnn-synthetic-offset.h5"
DATA_FOLDER = "build/window_split/real-data/1742552872"
SAVE_FILENAME = "cnn-retrained.h5"
LEARNING_RATE = 0.000029335291375872573

CNNSoftStartExecution.execute(MODEL_FILE, DATA_FOLDER, SAVE_FILENAME, LEARNING_RATE)

In [ ]:
from src.experiment import MultipleCNNTrainingExperiment

In [ ]:
MultipleCNNTrainingExperiment.start(10, 
                                    0.2, 
                                    "build/tune/sim_data_preprocessed_offset_multiplitiv_with_rot_forces/1743750402/syn-off-mul-rot.h5", 
                                    "assets/stationary_data_250403_01/sim_data_preprocessed_offset_multiplitiv_with_rot_forces.csv",
                                    0.0015009509278527317, 
                                    True,
                                    ";",
                                    ".", 
                                    326, 
                                    batch_split=True)

In [ ]:
from src.experiment import MultipleCNNValidationExperiment

MultipleCNNValidationExperiment.start("build/train/real", 10, make_validation=True)

In [2]:
from src.experiment import SoftStartExperiment

2025-04-04 10:50:41.138822: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-04 10:50:41.142396: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-04 10:50:41.152935: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743756641.170608   32120 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743756641.175672   32120 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-04 10:50:41.192956: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [14]:
MODEL_FILE = "build/train/syn-off-mul/1743673354/cnn.h5"
DATA_FOLDER = "build/window_split/real-data/1743683535"
SAVE_FILENAME = "re-syn-off-mul.h5"
LR = 0.0005110140569138342
LEARNING_RATES = [LR * 0.1]
Ns = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] # Number of epochs for the soft start



SoftStartExperiment.run(MODEL_FILE, DATA_FOLDER, SAVE_FILENAME, LEARNING_RATES, Ns)

Testing with learning rate: 5.110140569138342e-05, unfreezed layers: 1


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 10, 11)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 10, 13)    │        585 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_4     │ (None, 5, 13)     │          0 │ conv1d_4[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 65)        │          0 │ max_pooling1d_4[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 416)       │     27,456 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 384)       │    160,128 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 160)       │     61,600 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 480)       │     77,280 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 128)       │     61,568 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Verstellweg_X       │ (None, 1)         │        129 │ dense_8[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Verstellweg_Y       │ (None, 1)         │        129 │ dense_8[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Verstellweg_Phi     │ (None, 1)         │        129 │ dense_8[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 389,006 (1.48 MB)

 Trainable params: 389,004 (1.48 MB)

...

 Trainable params: 389,004 (1.48 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

None
Epoch 1/30
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 41s 6ms/step - Verstellweg_Phi_loss: 1.1029 - Verstellweg_Phi_mae: 1.1029 - Verstellweg_X_loss: 0.0332 - Verstellweg_X_mae: 0.0332 - Verstellweg_Y_loss: 0.9352 - Verstellweg_Y_mae: 0.9352 - loss: 2.3110 - val_Verstellweg_Phi_loss: 1.1991 - val_Verstellweg_Phi_mae: 1.2004 - val_Verstellweg_X_loss: 0.0343 - val_Verstellweg_X_mae: 0.0343 - val_Verstellweg_Y_loss: 0.6596 - val_Verstellweg_Y_mae: 0.6593 - val_loss: 2.1539
Epoch 2/30
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 38s 6ms/step - Verstellweg_Phi_loss: 0.4583 - Verstellweg_Phi_mae: 0.4583 - Verstellweg_X_loss: 0.0046 - Verstellweg_X_mae: 0.0046 - Verstellweg_Y_loss: 0.2190 - Verstellweg_Y_mae: 0.2190 - loss: 0.9477 - val_Verstellweg_Phi_loss: 0.9774 - val_Verstellweg_Phi_mae: 0.9784 - val_Verstellweg_X_loss: 0.0328 - val_Verstellweg_X_mae: 0.0328 - val_Verstellweg_Y_loss: 0.7478 - val_Verstellweg_Y_mae: 0.7473 - val_loss: 2.0358
Epoch 3/30
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 38s 6ms/step - Verstellweg_

build/soft-start/cnn/1743762212/re-syn-off-mul.h5


In [12]:
from src.experiment import MultipleCNNValidationExperiment

In [18]:
MODEL_FOLDER = "build/train/syn-off-mul-rot"
N = 10
MAKE_VALIDATION = True
DATA_FOLDER = "build/window_split/real-data/1743522733"

MultipleCNNValidationExperiment.start(MODEL_FOLDER, N, MAKE_VALIDATION, DATA_FOLDER)

3189/3189 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - Verstellweg_Phi_loss: 3.9833 - Verstellweg_Phi_mean_absolute_error: 3.9833 - Verstellweg_X_loss: 0.2314 - Verstellweg_X_mean_absolute_error: 0.2314 - Verstellweg_Y_loss: 5.7389 - Verstellweg_Y_mean_absolute_error: 5.7389 - loss: 10.1745


3189/3189 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - Verstellweg_Phi_loss: 3.7593 - Verstellweg_Phi_mean_absolute_error: 3.7593 - Verstellweg_X_loss: 0.2186 - Verstellweg_X_mean_absolute_error: 0.2186 - Verstellweg_Y_loss: 8.0774 - Verstellweg_Y_mean_absolute_error: 8.0774 - loss: 12.2788


3189/3189 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - Verstellweg_Phi_loss: 3.3876 - Verstellweg_Phi_mean_absolute_error: 3.3876 - Verstellweg_X_loss: 0.2305 - Verstellweg_X_mean_absolute_error: 0.2305 - Verstellweg_Y_loss: 9.6780 - Verstellweg_Y_mean_absolute_error: 9.6780 - loss: 13.5120


3189/3189 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - Verstellweg_Phi_loss: 3.1024 - Verstellweg_Phi_mean_absolute_error: 3.1024 - Verstellweg_X_loss: 0.2169 - Verstellweg_X_mean_absolute_error: 0.2169 - Verstellweg_Y_loss: 5.7094 - Verstellweg_Y_mean_absolute_error: 5.7094 - loss: 9.2460


3189/3189 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - Verstellweg_Phi_loss: 4.0657 - Verstellweg_Phi_mean_absolute_error: 4.0657 - Verstellweg_X_loss: 0.2178 - Verstellweg_X_mean_absolute_error: 0.2178 - Verstellweg_Y_loss: 8.3212 - Verstellweg_Y_mean_absolute_error: 8.3212 - loss: 12.8213


3189/3189 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - Verstellweg_Phi_loss: 3.0077 - Verstellweg_Phi_mean_absolute_error: 3.0077 - Verstellweg_X_loss: 0.2180 - Verstellweg_X_mean_absolute_error: 0.2180 - Verstellweg_Y_loss: 7.1818 - Verstellweg_Y_mean_absolute_error: 7.1818 - loss: 10.6418


3189/3189 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - Verstellweg_Phi_loss: 3.5421 - Verstellweg_Phi_mean_absolute_error: 3.5421 - Verstellweg_X_loss: 0.2194 - Verstellweg_X_mean_absolute_error: 0.2194 - Verstellweg_Y_loss: 6.9795 - Verstellweg_Y_mean_absolute_error: 6.9795 - loss: 10.9596


3189/3189 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - Verstellweg_Phi_loss: 2.9472 - Verstellweg_Phi_mean_absolute_error: 2.9472 - Verstellweg_X_loss: 0.2258 - Verstellweg_X_mean_absolute_error: 0.2258 - Verstellweg_Y_loss: 8.4348 - Verstellweg_Y_mean_absolute_error: 8.4348 - loss: 11.8349


3189/3189 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - Verstellweg_Phi_loss: 3.3935 - Verstellweg_Phi_mean_absolute_error: 3.3935 - Verstellweg_X_loss: 0.2210 - Verstellweg_X_mean_absolute_error: 0.2210 - Verstellweg_Y_loss: 8.1073 - Verstellweg_Y_mean_absolute_error: 8.1073 - loss: 11.9486


3189/3189 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - Verstellweg_Phi_loss: 3.9667 - Verstellweg_Phi_mean_absolute_error: 3.9667 - Verstellweg_X_loss: 0.2246 - Verstellweg_X_mean_absolute_error: 0.2246 - Verstellweg_Y_loss: 6.5960 - Verstellweg_Y_mean_absolute_error: 6.5960 - loss: 11.0081
Verstellweg_Phi_loss                    3.505654
Verstellweg_Phi_mean_absolute_error     3.505764
Verstellweg_X_loss                      0.217971
Verstellweg_X_mean_absolute_error       0.217933
Verstellweg_Y_loss                      7.458068
Verstellweg_Y_mean_absolute_error       7.457975
loss                                   11.403841
dtype: float64


In [ ]:
import keras

In [ ]:
m =  keras.models.load_model("build/tune/sim_data_preprocessed_offset_multiplitiv/1743667824/best-model.h5")
print(m.summary())